In [6]:
pip install paddleocr paddlepaddle opencv-python openai --upgrade -f https://www.paddlepaddle.org.cn/whl/paddleocr.html


Looking in links: https://www.paddlepaddle.org.cn/whl/paddleocr.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 1.7 MB/s eta 0:00:00-:--:--
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0.80
    Uninstalling opencv-python-4.9.0.80:
      Successfully uninstalled opencv-python-4.9.0.80
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from paddleocr import PaddleOCR
import cv2
import numpy as np
from PIL import Image

# Step 1: Load image and initialize OCR
img_path = "image_1007.jpg"
ocr = PaddleOCR(use_angle_cls=True, lang='en')
result = ocr.ocr(img_path, cls=True)

# Step 2: Read image with OpenCV
image = cv2.imread(img_path)
mask = np.zeros(image.shape[:2], dtype=np.uint8)  # start with a black mask

# Step 3: Draw white-filled polygons on mask for all detected text boxes
for line in result[0]:
    points = np.array(line[0]).astype(np.int32)
    cv2.fillPoly(mask, [points], 255)  # white area marks text

# Optional: visualize where the text is
# cv2.imwrite("text_mask.jpg", mask)

# Step 4: Inpaint (remove text)
inpainted = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
cv2.imwrite("cleaned_image.jpg", inpainted)


/Users/fatihwolf/miniconda3/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /Users/fatihwolf/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████████████████████████████████| 3910/3910 [00:20<00:00, 192.61it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /Users/fatihwolf/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|████████████████████████████████████| 10000/10000 [00:24<00:00, 406.87it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /Users/fatihwolf/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████████████████████████████████| 2138/2138 [00:15<00:00, 135.65it/s]

[2025/05/17 17:07:33] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/fatihwolf/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/fatihwolf/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_b

[2025/05/17 17:07:34] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.1284339427947998
[2025/05/17 17:07:34] ppocr DEBUG: cls num  : 5, elapsed : 0.017378807067871094
[2025/05/17 17:07:35] ppocr DEBUG: rec_res num  : 5, elapsed : 1.1412088871002197


True

In [ ]:
import cv2
import numpy as np
from paddleocr import PaddleOCR
import openai
from PIL import Image

# === Set up OpenAI client ===
from openai import OpenAI
client = OpenAI(api_key="")  # Replace with your key

# === Load image ===
img_path = "image_1007.jpg"
image = cv2.imread(img_path)

# === Run OCR ===
ocr = PaddleOCR(use_angle_cls=True, lang='en')
result = ocr.ocr(img_path, cls=True)

# === Extract text for GPT ===
detected_texts = [line[1][0] for line in result[0]]
full_text = " ".join(detected_texts)
print("🧠 Detected:", full_text)

# === Send to GPT-4o ===
comparison_prompt = f"""
You are an assistant that rewrites meme captions to be suitable for children under 13 years old.
Keep the message fun but remove any inappropriate or mature language.

Original: {full_text}

Kid-Friendly version:
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": comparison_prompt}],
    max_tokens=150,
    temperature=0.8
)

cleaned_caption = response.choices[0].message.content.strip()
print("🧼 Cleaned Caption:", cleaned_caption)

# === Create mask from text bounding boxes ===
mask = np.zeros(image.shape[:2], dtype=np.uint8)
for line in result[0]:
    points = np.array(line[0]).astype(np.int32)
    cv2.fillPoly(mask, [points], 255)

# === Inpaint ===
inpainted = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
cv2.imwrite("cleaned_image.jpg", inpainted)
print("✅ Inpainted image saved.")

# === Generate HTML output ===
html = f"""
<!DOCTYPE html>
<html>
<head><meta charset="UTF-8"><title>Meme for Kids</title></head>
<body style="text-align:center; font-family:sans-serif;">
  <h2>Cleaned Meme</h2>
  <img src="cleaned_image.jpg" width="500"/><br/><br/>
  <p><strong>Caption:</strong> {cleaned_caption}</p>
</body>
</html>
"""

with open("meme_output.html", "w", encoding="utf-8") as f:
    f.write(html)

print("📄 HTML saved as meme_output.html")


[2025/05/17 17:13:28] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/fatihwolf/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/fatihwolf/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_b

In [5]:
import base64
import io
from PIL import Image

# Convert inpainted image to PIL and then to base64
inpainted_rgb = cv2.cvtColor(inpainted, cv2.COLOR_BGR2RGB)
pil_img = Image.fromarray(inpainted_rgb)
buffered = io.BytesIO()
pil_img.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

# Use in HTML
html = f"""
<!DOCTYPE html>
<html>
<head><meta charset="UTF-8"><title>Meme for Kids</title></head>
<body style="text-align:center; font-family:sans-serif;">
  <h2>Cleaned Meme</h2>
  <img src="data:image/jpeg;base64,{img_str}" width="500"/><br/><br/>
  <p><strong>Caption:</strong> {cleaned_caption}</p>
</body>
</html>
"""

with open("meme_output.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ HTML with embedded image saved.")


✅ HTML with embedded image saved.


Below is the latest version it takes the images detects the text and than asks gpt partially for detection of inappropriateness and than retruns safe or what to change. if there is change needed we put a black box where the text is at and than rewrite the clean version.

In [ ]:
import cv2
import numpy as np
from paddleocr import PaddleOCR
from openai import OpenAI

# === Setup OpenAI client ===
client = OpenAI(api_key="")  # ← 🔑 Replace with your actual key

# === Load image ===
img_path = "image_1057.jpg"
image = cv2.imread(img_path)

# === Run OCR ===
ocr = PaddleOCR(use_angle_cls=True, lang='en')
result = ocr.ocr(img_path, cls=True)

def black_box_and_replace(image, box, cleaned_text, font_scale=1.0, thickness=2):
    pts = np.array(box).astype(np.int32)
    x1 = min([p[0] for p in pts])
    y1 = min([p[1] for p in pts])
    x2 = max([p[0] for p in pts])
    y2 = max([p[1] for p in pts])

    # Draw solid black rectangle
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 0), thickness=-1)

    # Calculate font size to fit the box
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size, _ = cv2.getTextSize(cleaned_text, font, font_scale, thickness)
    text_width, text_height = text_size

    # Adjust font size to fit the box
    max_width = x2 - x1 - 10
    while text_width > max_width and font_scale > 0.3:
        font_scale -= 0.05
        text_size, _ = cv2.getTextSize(cleaned_text, font, font_scale, thickness)
        text_width, text_height = text_size

    # Center text in box
    text_x = x1 + (x2 - x1 - text_width) // 2
    text_y = y1 + (y2 - y1 + text_height) // 2

    # Draw white text on black box
    cv2.putText(image, cleaned_text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

    return image

# === Process Each OCR Text Box ===
for line in result[0]:
    box = line[0]
    text = line[1][0].strip()

    if len(text) < 2:
        continue  # skip short garbage

    prompt = f"""
You are a content moderation assistant for meme captions.

Your job is to check if the input text is inappropriate for children under 13.

If the input **contains inappropriate language**, rewrite it into a **natural, clean, and child-appropriate version** that:
- Preserves the original intent or emotion (e.g., frustration, surprise, humor)
- Uses **real and understandable words**
- Avoids made-up substitutes or censored spellings (e.g., no asterisks or partial words)
- Uses normal sentence structure (not dot-separated words)

Keep the new version **similar in length** to the original so it fits visually in the image.

If the original text is already appropriate for children, reply with just: SAFE

Text: {text}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=60
        )
        reply = response.choices[0].message.content.strip()
    except Exception as e:
        print(f"⚠️ GPT failed on '{text}': {e}")
        continue

    print(f"🧠 Original: {text} → GPT Reply: {reply}")

    # === Filter safe results ===
    if reply.upper() != "SAFE":
        cleaned_text = reply.replace('"', '').replace("...", " ").replace("..", " ").replace(".", " ").strip()
        cleaned_text = cleaned_text.capitalize()
        image = black_box_and_replace(image, box, cleaned_text)

# === Save Final Output ===
cv2.imwrite("partially_cleaned_image2.jpg", image)
print("✅ Saved: partially_cleaned_image2.jpg")


[2025/05/17 18:03:53] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/fatihwolf/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/fatihwolf/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_b